MediGuard: AI Multi-Agent Medication & Safety Assistant

Architecture Diagram

User/Caregiver
      |
      v
[ Orchestrator Agent ]
      |
-------------------------------------------------
|   Prescription   Schedule   Instruction   Symptom
|     Intake       Builder    Simplifier     Log
-------------------------------------------------
                 |
                 v
     [ Backend Client / In-Memory Store ]

In [1]:
README.md

# MediGuard – AI-Powered Family Medication & Safety Assistant

A multi-agent AI system that helps families safely understand, organize, and track medications for elderly members, children, and patients with chronic conditions.

### 🏆 Google AI Agents Intensive – Capstone Project

---

## 🚨 Problem

Families often manage multiple prescriptions at the same time.  
This creates challenges:

- Difficult-to-read instructions  
- Forgotten medication timings  
- Forgotten doses  
- No easy way to track symptoms  
- Confusion about drug combinations  
- Overload for caregivers  

Mistakes lead to **missed doses**, **accidental double doses**, and unnecessary stress.

---

## 💡 Solution — MediGuard

**MediGuard** uses a team of AI agents to:

- Convert messy prescription text into **clean structured data**
- Generate personalized **daily medication schedules**
- Simplify complex instructions into **plain English**
- Log symptoms in structured format for **doctor visits**
- Highlight general **caution** about overlapping medications  
- Store everything through a backend client  
- Provide a clean “**Today’s Plan**” summary

MediGuard focuses on **clarity, safety, and support** — NOT diagnosis.

---

## 🧠 Multi-Agent Architecture

### Agents Used:

1. **Prescription Intake Agent**  
   Reads messy text → extracts JSON (drug, dose, frequency, instructions, etc.)

2. **Schedule Builder Agent**  
   Generates dose timings & sends them to backend.

3. **Instruction Simplifier Agent**  
   Converts medical jargon into normal language.

4. **Interaction Safety Agent**  
   Provides a cautious advisory note about overlapping effects  
   ("May cause drowsiness, dizziness… Consult a healthcare professional")

5. **Symptom Log Agent**  
   Summarizes caregiver symptom text and classifies severity (LOW/MEDIUM/HIGH).

6. **Orchestrator Agent**  
   Controls the entire flow using intents:
   - `add_prescription`
   - `log_symptom`
   - `today_plan`

---

## 🏗️ System Diagram

User/Caregiver
      |
      v
[ Orchestrator Agent ]
      |
-------------------------------------------------
|   Prescription   Schedule   Instruction   Symptom
|     Intake       Builder    Simplifier     Log
-------------------------------------------------
                 |
                 v
     [ Backend Client / In-Memory Store ]

---

## 🔧 Technical Implementation

MediGuard demonstrates all required Agent-Intensive concepts:

### ✔ Multi-Agent Collaboration  
### ✔ Orchestration  
### ✔ Structured Output (JSON)  
### ✔ Tool Use (Backend Client)  
### ✔ In-Memory Storage (Replaceable with Spring Boot)  
### ✔ Reasoning + Planning  

---



The notebook includes an **InMemoryBackendClient** that persists:

- Patients  
- Medications  
- Prescriptions  
- Dose schedules  
- Symptom logs  

For production, this can be replaced by a real **Spring Boot API**.

---

## 🚦 Safety Notes

MediGuard is a **prototype for educational purposes** ONLY.  
It does NOT:

- diagnose  
- prescribe  
- replace medical professionals  

Always consult a licensed healthcare provider.

---

## 🧪 Demo Scenarios

The notebook includes:

1. Add prescription  
2. Store medication schedule  
3. Log symptoms  
4. Retrieve today’s plan  
5. Receive general safety note  

---

## 🛠️ How to Run

1. Open Kaggle notebook  
2. Add Gemini key in  
   **Add-on → Secrets → GEMINI_API_KEY**  
3. Run all cells in order  
4. Try demo sections at the bottom

---

## 🚀 Future Improvements

- User authentication  
- Multiple caregivers  
- Reminders / WhatsApp integration  
- Charts for medication adherence  
- Cloud-run deployment  
- Replace in-memory backend with real **Spring Boot** backend


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 84)

In [ ]:
# ============================================================
#  Imports (run this first)
# ============================================================

import json
import datetime as dt
from dataclasses import dataclass, asdict
from typing import List, Dict, Any

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


In [ ]:
# ============================================================
#  Gemini setup with Kaggle Secrets
# ============================================================

user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise RuntimeError(
        "❌ No GEMINI_API_KEY found. Add it in Kaggle: Add-ons → Secrets → GEMINI_API_KEY."
    )

genai.configure(api_key=GEMINI_API_KEY)

MODEL_NAME = "gemini-2.0-flash"
model = genai.GenerativeModel(MODEL_NAME)

print("✅ Gemini client initialized.")


In [ ]:
# ============================================================
#  In-memory backend client (simulates Java backend)
# ============================================================

@dataclass
class Patient:
    id: int
    name: str


@dataclass
class Medication:
    id: int
    name: str
    strength: str


@dataclass
class Prescription:
    id: int
    patient_id: int
    medication_id: int
    dose: str
    frequency: str
    with_food: bool
    start_date: dt.date
    duration_days: int


@dataclass
class DoseSchedule:
    id: int
    prescription_id: int
    take_at: dt.datetime
    status: str = "PLANNED"


@dataclass
class SymptomLog:
    id: int
    patient_id: int
    logged_at: dt.datetime
    description: str
    severity: str  # LOW / MEDIUM / HIGH


class InMemoryBackendClient:
    def __init__(self):
        self._patients: Dict[int, Patient] = {}
        self._medications: Dict[int, Medication] = {}
        self._prescriptions: Dict[int, Prescription] = {}
        self._dose_schedules: Dict[int, DoseSchedule] = {}
        self._symptoms: Dict[int, SymptomLog] = {}

        self._patient_id = 1
        self._med_id = 1
        self._presc_id = 1
        self._dose_id = 1
        self._symptom_id = 1

    def get_or_create_patient(self, name: str) -> Patient:
        for p in self._patients.values():
            if p.name.lower() == name.lower():
                return p
        p = Patient(id=self._patient_id, name=name)
        self._patients[self._patient_id] = p
        self._patient_id += 1
        return p

    def get_or_create_medication(self, name: str, strength: str) -> Medication:
        for m in self._medications.values():
            if m.name.lower() == name.lower() and m.strength == strength:
                return m
        m = Medication(id=self._med_id, name=name, strength=strength)
        self._medications[self._med_id] = m
        self._med_id += 1
        return m

    def create_prescription(
        self,
        patient: Patient,
        medication: Medication,
        dose: str,
        frequency: str,
        with_food: bool,
        start_date: dt.date,
        duration_days: int,
    ) -> Prescription:
        p = Prescription(
            id=self._presc_id,
            patient_id=patient.id,
            medication_id=medication.id,
            dose=dose,
            frequency=frequency,
            with_food=with_food,
            start_date=start_date,
            duration_days=duration_days,
        )
        self._prescriptions[self._presc_id] = p
        self._presc_id += 1
        return p

    def generate_simple_schedule(
        self,
        prescription: Prescription,
        wake_time: dt.time = dt.time(7, 0),
        sleep_time: dt.time = dt.time(22, 0),
    ):
        freq = prescription.frequency.lower()
        doses_per_day = 1
        if "2" in freq:
            doses_per_day = 2
        elif "3" in freq:
            doses_per_day = 3

        if doses_per_day == 1:
            times = [wake_time.replace(hour=8)]
        elif doses_per_day == 2:
            times = [
                wake_time.replace(hour=8),
                sleep_time.replace(hour=20),
            ]
        else:
            times = [
                wake_time.replace(hour=8),
                wake_time.replace(hour=13),
                sleep_time.replace(hour=20),
            ]

        start = prescription.start_date
        end = start + dt.timedelta(days=prescription.duration_days)

        schedules = []
        current = start
        while current < end:
            for t in times:
                dt_obj = dt.datetime.combine(current, t)
                d = DoseSchedule(
                    id=self._dose_id,
                    prescription_id=prescription.id,
                    take_at=dt_obj,
                )
                self._dose_schedules[self._dose_id] = d
                schedules.append(d)
                self._dose_id += 1
            current += dt.timedelta(days=1)
        return schedules

    def log_symptom(self, patient: Patient, description: str, severity: str) -> SymptomLog:
        log = SymptomLog(
            id=self._symptom_id,
            patient_id=patient.id,
            logged_at=dt.datetime.now(),
            description=description,
            severity=severity,
        )
        self._symptoms[self._symptom_id] = log
        self._symptom_id += 1
        return log

    def get_todays_plan_for_patient(self, patient: Patient) -> Dict[str, Any]:
        today = dt.date.today()
        doses_today = []
        meds_today = []

        for d in self._dose_schedules.values():
            if d.take_at.date() == today:
                presc = self._prescriptions[d.prescription_id]
                if presc.patient_id == patient.id:
                    doses_today.append(d)
                    med = self._medications[presc.medication_id]
                    meds_today.append(med.name)

        meds_unique = sorted(set(meds_today))
        doses_info = []
        for d in sorted(doses_today, key=lambda x: x.take_at):
            presc = self._prescriptions[d.prescription_id]
            med = self._medications[presc.medication_id]
            doses_info.append(
                {
                    "time": d.take_at.strftime("%Y-%m-%d %H:%M"),
                    "medicine": med.name,
                    "strength": med.strength,
                    "dose": presc.dose,
                }
            )

        return {
            "patientName": patient.name,
            "date": today.isoformat(),
            "doses": doses_info,
            "medicines": meds_unique,
        }


backend = InMemoryBackendClient()
print("✅ In-memory backend ready.")


In [ ]:
# ============================================================
#  Utility helpers
# ============================================================

def pretty_json(obj: Any) -> str:
    return json.dumps(obj, indent=2, ensure_ascii=False,default=str)

def print_section(title: str, content: Any):
    print("\n" + "=" * 80)
    print(title)
    print("=" * 80)
    if isinstance(content, str):
        print(content)
    else:
        print(pretty_json(content))


In [ ]:
# ============================================================
#  Agent 1: Prescription Intake Agent
# ============================================================

def prescription_intake_agent(raw_text: str) -> Dict[str, Any]:
    prompt = f"""
You are a prescription intake assistant.

Extract:
- patientName
- medicineName
- strength (e.g. "500 mg")
- dose (e.g. "1 tablet")
- frequency (e.g. "2 times per day")
- withFood (true/false)
- startDate (YYYY-MM-DD, default today if missing)
- durationDays (int, default 30)

Text:
\"\"\"{raw_text}\"\"\"


Return ONLY valid JSON, no explanation, no backticks.
"""
    resp = model.generate_content(prompt)
    text = resp.text.strip().replace("```json", "").replace("```", "")
    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        data = {
            "patientName": "Unknown",
            "medicineName": "Unknown",
            "strength": "",
            "dose": "",
            "frequency": "",
            "withFood": False,
            "startDate": dt.date.today().isoformat(),
            "durationDays": 30,
            "raw": raw_text,
            "parseError": text,
        }
    return data


In [ ]:
# ============================================================
#  Agent 2: Schedule Builder Agent
# ============================================================

def schedule_builder_agent(structured_rx: Dict[str, Any]) -> Dict[str, Any]:
    start_date = dt.date.fromisoformat(structured_rx["startDate"])

    patient = backend.get_or_create_patient(structured_rx["patientName"])
    med = backend.get_or_create_medication(
        structured_rx["medicineName"],
        structured_rx.get("strength", ""),
    )

    presc = backend.create_prescription(
        patient=patient,
        medication=med,
        dose=structured_rx["dose"],
        frequency=structured_rx["frequency"],
        with_food=bool(structured_rx.get("withFood", False)),
        start_date=start_date,
        duration_days=int(structured_rx["durationDays"]),
    )

    schedules = backend.generate_simple_schedule(presc)
    sample = [
        {
            "time": s.take_at.strftime("%Y-%m-%d %H:%M"),
            "status": s.status,
        }
        for s in schedules[:6]
    ]

    return {
        "patient": asdict(patient),
        "medication": asdict(med),
        "prescription": asdict(presc),
        "scheduleCount": len(schedules),
        "sampleSchedule": sample,
    }


In [ ]:
# ============================================================
#  Agent 3: Instruction Simplifier Agent
# ============================================================

def instruction_simplifier_agent(raw_text: str) -> str:
    prompt = f"""
Rewrite these medication instructions in simple, clear language
for a non-medical person. Use short sentences.

Text:
\"\"\"{raw_text}\"\"\""""
    resp = model.generate_content(prompt)
    return resp.text.strip()


In [ ]:
# ============================================================
#  Agent 4: Interaction Safety Agent
# ============================================================

def interaction_safety_agent(medicine_names: List[str]) -> str:
    if not medicine_names:
        return "No medicines found for this patient today."

    prompt = f"""
You are a cautious medication interaction explainer.

Medicines for this patient today:
{medicine_names}

- Look for possible overlapping side effects (sedation, dizziness, bleeding, etc.).
- Do NOT say there is a definite interaction.
- Do NOT tell the user to stop or change medication.
- Use simple language.
- Always end with: "Please consult a licensed healthcare professional before making any changes."

Respond in plain English.
"""
    resp = model.generate_content(prompt)
    return resp.text.strip()


In [ ]:
# ============================================================
#  Agent 5: Symptom Log Agent
# ============================================================

def symptom_log_agent(patient_name: str, symptom_text: str) -> Dict[str, Any]:
    prompt = f"""
You are a symptom summarizer.

Patient: "{patient_name}"
Caregiver says:
\"\"\"{symptom_text}\"\"\"


1. Write a one-sentence summary.
2. Classify severity as LOW, MEDIUM, or HIGH.
Return ONLY JSON as:
{{
  "summary": "...",
  "severity": "LOW" | "MEDIUM" | "HIGH"
}}
"""
    resp = model.generate_content(prompt)
    text = resp.text.strip().replace("```json", "").replace("```", "")
    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        data = {"summary": symptom_text, "severity": "MEDIUM", "raw": text}

    patient = backend.get_or_create_patient(patient_name)
    log = backend.log_symptom(patient, data["summary"], data["severity"])
    return {"analysis": data, "storedLog": asdict(log)}


In [ ]:
# ============================================================
#  Agent 6: Orchestrator Agent 
# ============================================================

def medi_guard_orchestrator(intent: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    """
    Central orchestrator that routes a high-level request to the
    appropriate sub-agents.

    Supported intents:
      - "add_prescription": payload["text"]
      - "log_symptom": payload["patientName"], payload["symptomText"]
      - "today_plan": payload["patientName"]
    """
    intent = intent.lower()

    # --------------------------------------------------------
    # 1) Add a new prescription + build schedule
    # --------------------------------------------------------
    if intent == "add_prescription":
        raw = payload["text"]

        # 1. Parse the free-text prescription
        structured = prescription_intake_agent(raw)

        # 🔧 DEMO FIX:
        # Force the schedule to start today so Demo 3 ("today_plan")
        # actually finds doses for the current date.
        structured["startDate"] = dt.date.today().isoformat()

        # Optional: keep the demo small (e.g., 7 days instead of 30+)
        if "durationDays" not in structured or not structured["durationDays"]:
            structured["durationDays"] = 7
        else:
            # make sure it's an int and cap it for demo
            try:
                structured["durationDays"] = min(int(structured["durationDays"]), 14)
            except Exception:
                structured["durationDays"] = 7

        # 2. Generate schedule & persist via backend
        schedule_summary = schedule_builder_agent(structured)

        # 3. Simplify the original instructions for the caregiver
        simplified = instruction_simplifier_agent(raw)

        return {
            "structuredPrescription": structured,
            "scheduleSummary": schedule_summary,
            "simplifiedInstruction": simplified,
        }

    # --------------------------------------------------------
    # 2) Log a symptom for a patient
    # --------------------------------------------------------
    if intent == "log_symptom":
        patient_name = payload["patientName"]
        symptom_text = payload["symptomText"]
        return symptom_log_agent(patient_name, symptom_text)

    # --------------------------------------------------------
    # 3) Get today's plan + safety note
    # --------------------------------------------------------
    if intent == "today_plan":
        patient_name = payload["patientName"]

        # Ensure patient exists in backend (same helper used elsewhere)
        patient = backend.get_or_create_patient(patient_name)

        # Get all doses scheduled for *today*
        plan = backend.get_todays_plan_for_patient(patient)

        # Generate a cautious safety note about today's medicines
        safety_note = interaction_safety_agent(plan.get("medicines", []))

        return {
            "todayPlan": plan,
            "safetyNote": safety_note,
        }

    # --------------------------------------------------------
    # Fallback for unknown intents
    # --------------------------------------------------------
    return {"error": f"Unknown intent '{intent}'"}



In [ ]:
# ============================================================
#  Demo 1: Add prescription
# ============================================================

demo_prescription_text = """
Patient: Grandma
Medicine: Metformin 500 mg tablets
Take 1 tablet twice daily with food.
Start today and continue for 30 days.
"""

result_demo1 = medi_guard_orchestrator(
    intent="add_prescription",
    payload={"text": demo_prescription_text},
)

print_section("Structured Prescription", result_demo1["structuredPrescription"])
print_section("Schedule Summary", result_demo1["scheduleSummary"])
print_section("Simplified Instruction", result_demo1["simplifiedInstruction"])


In [ ]:
# ============================================================
#  Demo 2: Log symptom
# ============================================================

demo_symptom_text = """
Grandma feels a bit dizzy about one hour after taking her morning tablet.
"""

result_demo2 = medi_guard_orchestrator(
    intent="log_symptom",
    payload={
        "patientName": "Grandma",
        "symptomText": demo_symptom_text,
    },
)

print_section("Symptom Analysis & Stored Log", result_demo2)


In [ ]:
# ============================================================
#  Demo 3: Today's plan & safety note
# ============================================================

result_demo3 = medi_guard_orchestrator(
    intent="today_plan",
    payload={"patientName": "Grandma"},
)

print_section("Today's Medication Plan", result_demo3["todayPlan"])
print_section("Safety Note", result_demo3["safetyNote"])
